In [1]:
import pandas as pd
import xlwings as xw   #Load password protected Excel files
import numpy as np
import matplotlib.pyplot as plt
import sys
import codecs
import datetime


PATH = "NTU深耕客服資料檔_pass_769.xlsx"
wb = xw.Book(PATH)
sheet = wb.sheets['sample']

df = sheet['A1:D1000'].options(pd.DataFrame, index=False, header=True).value #檔案必須是打開的狀態
id_df = df.sort_values(['Unique ID', '資料日期'])
id_df_comb = id_df.groupby('Unique ID')['客戶事件描述'].apply(' '.join)
unique_id = pd.DataFrame({'Unique ID': id_df_comb.index})

start = datetime.datetime(2018, 1, 1)
end = datetime.datetime(2018, 12, 31)
days = pd.date_range(start, end, freq='D')
days = days.strftime('%Y%m%d')

index = unique_id['Unique ID']
columns = days
output = pd.DataFrame(index = index, columns = columns)

date = id_df["資料日期"]
for i in range(len(date)):
	id_df["資料日期"][i] = id_df["資料日期"][i][0:8]

for i in range(len(id_df['資料日期'])):
	date = id_df['資料日期'][i]
	desc = id_df['客戶事件描述'][i]
	ID = id_df['Unique ID'][i]
	if(pd.isnull(output.loc[ID, date])):
		output.loc[ID, date] = "" 
	output.loc[ID, date] += (desc+"\n")

#print(output)
remember = output

In [2]:
#---------WEEK-------------------------------------------------
for i in range(len(output.index)):
	for j in range(0, len(output.columns), 7):
		if j > 364:
			break
		tmp = ""
		for k in range(7):
			if j+k > 364:
				break
			if pd.isnull(output.loc[output.index[i], output.columns[j+k]]):
				continue
			tmp += (str(output.loc[output.index[i], output.columns[j+k]]) + " ")
		#print(tmp)
		output.loc[output.index[i], output.columns[j]] = tmp


drop_list = []
for i in range(len(output.columns)):
	if i%7 == 0:
		continue
	drop_list.append(output.columns[i])
	
output = output.drop(drop_list, axis=1)
output.to_csv('week.csv',encoding='utf_8_sig')
output.head()

,20180101,20180108,20180115,20180122,20180129,20180205,20180212,20180219,20180226,20180305,...,20181029,20181105,20181112,20181119,20181126,20181203,20181210,20181217,20181224,20181231
Unique ID,,,,,,,,,,,,,,,,,,,,,
AA0063,,,,,,,,,,,...,,,,,,,,,,
AA0102,,,,,,,,,,,...,,,,,,,,,,
AA0263,,有進線問IPO.但沒有要理專回電 用mail \n,,[ECP]IPO 美元 有機會 \n,,,,,,,...,,,,,,,,,,
AA0269,,,,,,,[ECP]KYC名單 客戶說不需要\n,,,,...,,,,,,,,,,
AA0305,,,,,,,,,,,...,,,,,,,,,,


In [3]:
#---------MONTH-------------------------------------------------
month_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
output = remember
no_drop = [0] + month_days[0:-1]
no_drop_list = []
for i in range(12):
	no_drop_list.append(sum(no_drop[0:i+1]))
#print(no_drop_list)
drop_list = list(range(0, 365))
drop_list = [x for x in drop_list if x not in no_drop_list]
#print(drop_list)

#drop_date = []
#for i in range(len(drop_list)):
drop_date = list(output.columns[drop_list])
#print(drop_date)




for i in range(len(output.index)):
	for j in range(12):
		tmp = ""
		for k in range(month_days[j]):
			if pd.isnull(output.loc[output.index[i], output.columns[sum(month_days[0:j]) + k]]):
				continue
			tmp += (str(output.loc[output.index[i], output.columns[sum(month_days[0:j]) + k]]) + " ")
			output.loc[output.index[i], output.columns[sum(month_days[0:j])]] = tmp



	
output = output.drop(drop_date, axis=1)
output.to_csv('month.csv',encoding='utf_8_sig')
output.head()

,20180101,20180201,20180301,20180401,20180501,20180601,20180701,20180801,20180901,20181001,20181101,20181201
Unique ID,,,,,,,,,,,,
AA0063,,,,,[ECP]泰國巴西名單 客戶說在打球 先mail 2Q優惠\n [ECP]泰國巴西名單...,,,,,,,
AA0102,,,推Q1，家庭戶 \n 推Q1，家庭戶 \n,,,,,,,,,
AA0263,有進線問IPO.但沒有要理專回電 用mail \n 有進線問IPO.但沒有要理專回電 ...,,,,,已開ipo edm 再mail\n 已開ipo edm 再mail\n,[ECP]可意買亞債\n [ECP]可意買亞債\n,,,,,
AA0269,,[ECP]KYC名單 客戶說不需要\n [ECP]KYC名單 客戶說不需要\n,,,,,,,,,,
AA0305,,,,,,,,,[ECP]家庭戶 徐O 有問到被動原件\n [ECP]家庭戶 徐O 有問到被動原件\n,,,


In [4]:
#---------Quarter-------------------------------------------------
for i in range(len(output.index)):
	for j in range(0, 12, 3):
		tmp = ""
		for k in range(3):
			if pd.isnull(output.loc[output.index[i], output.columns[j + k]]):
				continue
			tmp += (str(output.loc[output.index[i], output.columns[j + k]]))
			output.loc[output.index[i], output.columns[j]] = tmp


quarter_drop = []
for i in range(12):
	if i%3 == 0:
		continue
	quarter_drop.append(output.columns[i])
#print(quarter_drop)


output = output.drop(quarter_drop, axis=1)
output.to_csv('quarter.csv',encoding='utf_8_sig')
output.head()

,20180101,20180401,20180701,20181001
Unique ID,,,,
AA0063,,[ECP]泰國巴西名單 客戶說在打球 先mail 2Q優惠\n [ECP]泰...,,
AA0102,推Q1，家庭戶 \n 推Q1，家庭戶 \n,,,
AA0263,有進線問IPO.但沒有要理專回電 用mail \n 有進線問IPO.但沒有要理專回電 ...,已開ipo edm 再mail\n 已開ipo edm 再mail\n,[ECP]可意買亞債\n [ECP]可意買亞債\n,
AA0269,[ECP]KYC名單 客戶說不需要\n [ECP]KYC名單 客戶說不需要\n...,,,
AA0305,,,[ECP]家庭戶 徐O 有問到被動原件\n [ECP]家庭戶 徐O 有問...,
